In [ ]:
from random import randint, choices
from string import digits
import pandas as pd
#from faker import Faker  # 가짜 이름 라이브러리


class WithdrawError(Exception):  # 출금 에러 클래스
    def __init__(self, account, amount):
        self.account = account
        self.amount = amount

    def __str__(self):
        return f'insufficient balance: ' \
               f'balance is ￦{self.account.get_balance():,d} ' \
               f'but withdrawal amount is ￦{self.amount:,d}'


class PaymentError(Exception):
    def __init__(self, payment):
        self.payment = payment

    def __str__(self):
        return f'invalid payment method: {self.payment}'


class BankAccount:
    def __init__(self, name):
        self.__account_number = ''.join(choices(digits, k=13))
        self.__balance = 0  # 잔액
        self.name = name

    def __str__(self):
        return '이름: {:s}\n' \
               '계좌번호: {:0>4s}-{:s}-{:s}\n' \
               '잔액: ￦{:,d}'.format(
                self.name,
                self.__account_number[:-9],
                self.__account_number[-9:-7],
                self.__account_number[-7:],
                self.__balance)

    def get_balance(self):
        return self.__balance

    def deposit(self, amount):  # 입금
        self.__balance += amount

    def withdraw(self, amount):  # 출금
        if amount <= self.__balance:
            self.__balance -= amount
        else:
            raise WithdrawError(self, amount)


class Card:
    def __init__(self, account):
        self.__card_number = ''.join(choices(digits, k=16))
        self.__history = []  # 카드 사용 내역
        self.account = account  # 카드 소유주

    def __str__(self):
        return '카드번호: {}\n' \
               '↓ 소유주 정보 ↓\n' \
               '{}'.format(
                '-'.join([self.__card_number[i:i + 4] for i in range(0, 16, 4)]),
                self.account)


class VendingMachine:  # 자판기 클래스
    def __init__(self):
        item_list = [
            '글루텐-프리 에너지바',
            '글루텐-프리 어니언링',
            '글루텐-프리 크래커',
            '글루텐-프리 시리얼',
            '글루텐-프리 베이글',
            '글루텐-프리 감자칩',
            '글루텐-프리 쿠키',
            '할랄 소시지',
            '할랄 치킨',
            '할랄 케밥',
            '할랄 파이',
            '할랄 캔디'
        ]
        self.__items = pd.DataFrame(
            {item: [randint(10, 20), randint(7, 20) * 100] for item in item_list},
            index=['수량', '가격'])
        self.__items = self.__items.transpose()
        self.__moneybox = {
            10000: 0,
            5000: 0,
            1000: 100,
            500: 200,
            100: 500
        }

    def __str__(self):
        return str(self.__items)

    def buy(self):
        payment = input('결제 수단을 입력하세요. (현금 또는 카드): ')
        if payment == '현금':
            self.__buy_with_cash()
        elif payment == '카드':
            self.__buy_with_card()
        else:
            raise PaymentError(payment)

    def __buy_with_cash(self):
        item = input("구매할 상품을 입력하세요: ")
        if item not in self.__items.index:
            print("잘못된 상품입니다.")
            return
        
        price = self.__items.loc[item, '가격']
        quantity = self.__items.loc[item, '수량']
        
        if quantity <= 0:
            print("재고가 없습니다.")
            return
        
        amount_given = int(input(f"상품 가격은 ￦{price:,d}입니다. 현금을 입력하세요: "))
        
        if amount_given < price:
            print("지불 금액이 부족합니다.")
            return
        
        change = amount_given - price
        
        if not self.__give_change(change):
            print("거스름돈이 부족합니다. 카드 결제만 가능합니다.")
            return
        
        self.__items.loc[item, '수량'] -= 1
        print(f"구매 완료! 거스름돈은 ￦{change:,d}입니다.")
        self.__print_receipt("현금", price, change)
        
    def __buy_with_card(self):
        item = input("구매할 상품을 입력하세요: ")
        if item not in self.__items.index:
            print("잘못된 상품입니다.")
            return
        
        price = self.__items.loc[item, '가격']
        quantity = self.__items.loc[item, '수량']
        
        if quantity <= 0:
            print("재고가 없습니다.")
            return
        
        card_number = input("카드 번호를 입력하세요: ")
        if card_number != card1.account.__str__():
            print("잘못된 카드 번호입니다.")
            return
        
        if card1.account.get_balance() < price:
            print("카드 잔액이 부족합니다.")
            return
        
        card1.account.withdraw(price)
        self.__items.loc[item, '수량'] -= 1
        print("구매 완료!")
        self.__print_receipt("카드", price)
    
    def __give_change(self, amount):
        change = {}
        for denom in sorted(self.__moneybox.keys(), reverse=True):
            coin_value = denom
            while amount >= coin_value and self.__moneybox[denom] > 0:
                if denom not in change:
                    change[denom] = 0
                change[denom] += 1
                self.__moneybox[denom] -= 1
                amount -= coin_value
        if amount == 0:
            return True
        else:
            # If we couldn't provide the exact change, roll back changes
            for denom in change:
                self.__moneybox[denom] += change[denom]
            return False
    
    def __print_receipt(self, payment_method, item_price, change_given=None):
        receipt = f"결제 방법: {payment_method}\n상품 가격: ￦{item_price:,d}\n"
        if payment_method == "현금" and change_given is not None:
            receipt += f"거스름돈: ￦{change_given:,d}\n"
        receipt += "감사합니다!\n"
        print(receipt)

    def admin_login(self, password):
        # 관리자 비밀번호는 'admin'으로 설정
        if password == 'admin':
            return True
        else:
            print("잘못된 비밀번호입니다.")
            return False

    def change_item(self, old_item, new_item, new_price, new_quantity):
        if old_item in self.__items.index:
            self.__items = self.__items.drop(index=old_item)
        self.__items.loc[new_item] = [new_quantity, new_price]
        print(f"상품 변경 완료: {old_item} -> {new_item}, 가격: ￦{new_price:,d}, 수량: {new_quantity}")

    def restock_item(self, item, quantity):
        if item in self.__items.index:
            self.__items.loc[item, '수량'] += quantity
            print(f"{item} 재고가 {quantity}개 추가되었습니다.")
        else:
            print("잘못된 상품입니다.")

    def restock_moneybox(self, money_type, quantity):
        if money_type in self.__moneybox:
            self.__moneybox[money_type] += quantity
            print(f"{money_type}원권 지폐/동전 {quantity}개가 추가되었습니다.")
        else:
            print("잘못된 화폐 단위입니다.")


# 자판기 예제 사용
#card1 = Card(BankAccount(Faker('ko-KR').name()))
#card1.account.deposit(randint(100, 999) * 1000)
#print(card1)

vending_machine1 = VendingMachine()
print(vending_machine1)

# 관리자 모드 예제
admin_password = input("관리자 비밀번호를 입력하세요: ")
if vending_machine1.admin_login(admin_password):
    vending_machine1.change_item('글루텐-프리 에너지바', '신규 글루텐-프리 에너지바', 1500, 30)
    vending_machine1.restock_item('할랄 케밥', 10)
    vending_machine1.restock_moneybox(1000, 50)

# 사용자 모드 예제
vending_machine1.buy()


             수량    가격
글루텐-프리 에너지바  18   800
글루텐-프리 어니언링  20  1700
글루텐-프리 크래커   11  1100
글루텐-프리 시리얼   14  1400
글루텐-프리 베이글   17  1700
글루텐-프리 감자칩   11  1100
글루텐-프리 쿠키    17  1200
할랄 소시지       10  2000
할랄 치킨        15  1500
할랄 케밥        20   900
할랄 파이        10   800
할랄 캔디        20  1900
관리자 비밀번호를 입력하세요: admin
상품 변경 완료: 글루텐-프리 에너지바 -> 신규 글루텐-프리 에너지바, 가격: ￦1,500, 수량: 30
할랄 케밥 재고가 10개 추가되었습니다.
1000원권 지폐/동전 50개가 추가되었습니다.
